In [1]:
#import cifar10
#import cifar10_input
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
import pickle

In [2]:
def _load_label_name():
    return ['airplane', 'automobile', ' bird', 'cat', 'dog',
            'frog', 'forse', 'ship' , 'truck']

In [12]:
def load_cfar10_batch(bacth_string,batch_id):
    if batch_string == 'train':
        batch_string1 = '/data_batch_'
        with open('cifar-10-batches-py'+batch_string1+str(batch_id),mode = 'rb') as file:
            batch = pickle.load(file, encoding = 'latin1')
    else:
        batch_string1 = '/test_batch'
        with open('cifar-10-batches-py'+batch_string1,mode = 'rb') as file:
            batch = pickle.load(file, encoding = 'latin1')
    features = batch['data'].reshape(len(batch['data']),3,32,32).transpose(0,2,3,1)
    labels = batch['labels']
    return features,labels

In [11]:
'asd' == 'asd'

True

In [6]:
def display(batch_id, sample_id):
    batch_ids = list(range(1,6))
    if batch_id not in batch_ids:
        print(' Batch Id out of Range. Possible Batch Ids: {}'.format(batch_ids))
        return None
    features , labels = load_cfar10_batch('train',batch_id)
    if not (0<=sample_id<len(features)):
        print('{}samples in batch {}. {} is out of range'.format(len(features),batch_id,sample_id))
        return None
    print('\nStats of batch {}:'.format(batch_id))
    print('Samples: {}'.format(len(features)))
    print('Label Counts: {}'.format(dict(zip(*np.unique(labels, return_counts = True)))))
    print('First 20 Labels: {}'.format(labels[:20]))
    sample_image = features[sample_id]
    sample_label = labels[sample_id]
    label_names = _load_label_name()
    print('\nExample of Image {}:'.format(sample_id))
    print('Image - Min Vlaue: {} Max Value: {}'.format(sample_image.min(),sample_image.max()))
    print('Image - Shape: {}'.format(sample_image.shape))
    print('Label - Label Id: {} Name: {}'.format(sample_label, label_names[sample_label]))
    plt.axis('off')
    plt.imshow(sample_image)

In [13]:
display(3,2)

NameError: name 'batch_string' is not defined

In [24]:
max_steps = 3000
batch_size = 128

data_dir = '/home/a503wangsiqi/mine/cifar_data/'
def variable_with_weights_loss(shape, stddev , w1):
    var = tf.Variable(tf.truncated_normal(shape , stddev=stddev))
    if w1 is not None:
        weights_loss = tf.multiply(tf.nn.l2_loss(var) , w1 , name = 'weight_loss')
        tf.add_to_collection('losses' , weights_loss)
    return  var

#cirfar10_maybe_download_and_extract()
#image_train, label_train = cifar10_input.distorted_inputs(data_dir = data_dir , batch_size =batch_size)
#image_test , label_test = cifar10_input.inputs(eval_data = True , data_dir = data_dir , batch_size =batch_size)

image_train, label_train = load_cfar10_batch('train',1)
image_test, label_test = load_cfar10_batch('test',1)


image_holder = tf.placeholder(tf.float32 , [batch_size , 24 , 24 , 3])
label_holder = tf.placeholder(tf.int32 , [batch_size])

weights1 = variable_with_weights_loss(shape= [5,5,3,64], stddev=5e-2 , w1 = 0.0)#不计入loss
kernel1 = tf.nn.conv2d(image_holder , weights1 , [1,1,1,1] , padding = 'SAME')
bias1 = tf.Variable(tf.constant(0.0 , shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1 , bias1))
pool1 = tf.nn.max_pool(conv1 , ksize = [ 1, 3, 3, 1] , strides= [1, 2, 2, 1], padding= 'SAME')

norm1 = tf.nn.lrn(pool1 , 4 , bias = 1.0 , alpha= 0.001/9.0, beta= 0.75)

weights2 = variable_with_weights_loss(shape = [5,5,64,64],stddev= 5e-2 , w1= 0.0)#不计入loss
kernel2 = tf.nn.conv2d(norm1 , weights2 , [1,1,1,1] , padding = 'SAME')
bias2 = tf.Variable(tf.constant(0.1 , shape = [64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2 , bias2))
norm2 = tf.nn.lrn(conv2 , 4 , bias=1.0, alpha = 0.001/9.0, beta= 0.75)
pool2 = tf.nn.max_pool(norm2 , ksize = [1,3,3,1] , strides=[1,2,2,1] , padding= 'SAME')

reshape = tf.reshape((pool2 , [batch_size, -1]))
dim = reshape.get_shape()[1].value
weights3 = variable_with_weights_loss(shape=[dim ,384], stddev= 0.04 , w1 = 0.004)#加入loss，防止全连接层过拟合
bias3 = tf.Variable(tf.constant(0.1 , shape= [384]))
local3 = tf.nn.relu(tf.matmul(reshape , weights3) + bias3)

weights4 = variable_with_weights_loss(shape= [384 , 192] , stddev= 0.04 , w1 = 0.004)#加入loss，防止全链接层过拟合
bias4 = tf.Variable(tf.constant(0.1 , shape= [192]))
local4 = tf.nn.relu(tf.matmul(local3 , weights4) + bias4)

weights5 = variable_with_weights_loss(shape = [192,10] , stddev= 1/192.0 ,w1 = 0.0)#不加入loss
bias5 = tf.Variable(tf.constant(0.1 ,shape= [10]))
logits = tf.add(tf.matmul(local4 , weights5) , bias5)

def loss(logits , labels):
    labels = tf.cast(labels, tf.float64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits= logits , labels= labels , name = 'cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy , name= 'cross_entropy')
    tf.add_to_collection('losses' , cross_entropy_mean)

    return tf.add_n(tf.get_collection('losses') , name = 'total_loss')

loss = loss(logits , label_holder)
train_op = tf.train.AdamOptimizer(0.001).minimize(loss)
top_k_op = tf.nn.in_top_k(logits , label_holder , 1)

sess = tf.InteractiveSession()
tf.global_variables_initializer()

tf.train.start_queue_runners()
for step in range(max_steps):
    start_time = time.time()
    image_batch,label_batch = sess.run([image_train,label_train])
    _ , loss_value = sess.run([train_op , loss] , feed_dict={image_holder:image_batch , label_holder:label_batch})
    #想同时运行两个tensor扩成一个list
    
    duration = time.time() - start_time
    if step % 10 ==0:
        example_per_sec = batch_size/duration
        sec_per_batch = float(duration)

        format_str = ('step %d , loss = %.2f(%.1f examples/sec ; %.3f sec/batch')
        print(format_str%(step,loss_value, example_per_sec,sec_per_batch))
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples/batch_size))
true_count =0
total_sample_count = num_iter * batch_size
step =0
while step<num_iter:
    image_batch ,label_batch = sess.run([image_test,label_test])
    predictions = sess.run([top_k_op],feed_dict={image_holder:image_batch,label_holder:label_batch})
    true_count += np.sum(predictions)
    step+=1
prediction = true_count / total_sample_count
print('prediction @ 1=%.3f'%prediction)

TypeError: load_cfar10_batch() takes 1 positional argument but 2 were given